## Amazon Lookout for Vision

### Overview

A simple application that uses [Amazon Lookout for Vision](https://aws.amazon.com/lookout-for-vision/) to spot defects and detect anomalies.

![Demo Application](images/demo_app.png)

### What is Amazon Lookout for Vision?

> *Amazon Lookout for Vision is a machine learning (ML) service that <mark>spots defects and anomalies in visual representations using computer vision (CV)</mark>. With Amazon Lookout for Vision, manufacturing companies can increase quality and reduce operational costs by quickly identifying differences in images of objects at scale. For example, Amazon Lookout for Vision can be used to identify missing components in products, damage to vehicles or structures, irregularities in production lines, miniscule defects in silicon wafers, and other similar problems. <mark>Amazon Lookout for Vision uses ML to see and understand images from any camera as a person would, but with an even higher degree of accuracy and at a much larger scale</mark>. Amazon Lookout for Vision allows customers to eliminate the need for costly and inconsistent manual inspection, while improving quality control, defect and damage assessment, and compliance. In minutes, you can begin using Amazon Lookout for Vision to automate inspection of images and objects–with no machine learning expertise required.*

For more information, please visit [Amazon Lookout for Vision](https://aws.amazon.com/lookout-for-vision/)

![Lookout for Vision](images/lookout_for_vision.png)

### Preparation

Define some constants

In [ ]:
AWS_DEFAULT_REGION = 'eu-central-1'
DEFAULT_IMAGES_FOLDER = 'extra_images'
DEFAULT_NOISY_FOLDER = 'noisy_images'

Add logging configuration

In [ ]:
import logging

logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s',
                    level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S')

Install requirements

In [ ]:
!pip install -q -r requirements.txt
!pip install -q -r requirements_jupyter.txt

Setup AWS credentials

In [ ]:
import os
from getpass import getpass

# Get AWS credentials
if(set(['AWS_ACCESS_KEY', 'AWS_SECRET_ACCESS_KEY', 'AWS_SECRET_ACCESS_TOKEN']) <= set(os.environ.keys())):
    logging.info("Detected AWS credentials")
else:
    os.environ['AWS_ACCESS_KEY'] = getpass('+ AWS_ACCESS_KEY: ')
    os.environ['AWS_SECRET_ACCESS_KEY'] = getpass('+ AWS_SECRET_ACCESS_KEY: ')
    os.environ['AWS_SECRET_ACCESS_TOKEN'] = getpass('+ AWS_SECRET_ACCESS_TOKEN: ')

# Set default AWS region
os.environ['AWS_DEFAULT_REGION'] = input("+ AWS_DEFAULT_REGION (%s): " % AWS_DEFAULT_REGION) or AWS_DEFAULT_REGION

### Data

Download extra images

In [ ]:
# Whether to download extra images
download_images = True

# Check if destination exists
if os.path.isdir(DEFAULT_IMAGES_FOLDER):
    logging.info("Destination path '{}' already exists.".format(DEFAULT_IMAGES_FOLDER))
    download_images = input("Continue (y/n)? ") == 'y'

# Download images from S3 bucket
if download_images:
    logging.info("Downloading extra images")
    os.system('aws s3 cp s3://circuitboarddataset/circuit_board/extra_images/ {}/ --recursive'.format(DEFAULT_IMAGES_FOLDER))

### Demo

Select a project, model version and the location of the extra images

In [ ]:
import boto3
import l4v_app

from glob import glob

# Initialize Lookout for Vision client
lookout_vision = boto3.client(
    'lookoutvision',
    aws_access_key_id=os.environ['AWS_ACCESS_KEY'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    aws_session_token=os.environ['AWS_SECRET_ACCESS_TOKEN'],
    region_name=os.environ['AWS_DEFAULT_REGION']
)

# Retrieve list of projects
projects = l4v_app.list_projects(lookout_vision)

# Get project name from user
project_name = input('+ Project Name %s: ' % projects) or projects[0]

# Retrieve list of models for the selected project
models = l4v_app.list_models(lookout_vision, project_name)

# Get project name from user
model_version = input('+ Model Version %s: ' % models) or models[0]

# Get images folder
images_folder = input('+ Images (%s): ' % DEFAULT_IMAGES_FOLDER) or DEFAULT_IMAGES_FOLDER

Display some information about the project

In [ ]:
import json

print(json.dumps(l4v_app.describe_project(lookout_vision, project_name), indent=4, sort_keys=True, default=str))

Is the model running?

In [ ]:
print(l4v_app.describe_model(lookout_vision, project_name, model_version)['StatusMessage'])

Test the model on a single image

In [ ]:
%timeit

args = [
    "--project", project_name,
    "--model", model_version,
    "--images", glob(images_folder + '/*')[0],
    "--wait", '10',
    "--display"  # comment to remove display
]
l4v_app.main(lookout_vision, args)

Run anomaly detection on all images and save the results

In [ ]:
%timeit

args = [
    "--project", project_name,
    "--model", model_version,
    "--images", images_folder,
    "--save", 'results_original.csv'
]

l4v_app.main(lookout_vision, args)

Load results from CSV file

In [ ]:
import pandas as pd

# Load and process results
results = pd.read_csv('results_original.csv')
results['prediction'] = results['prediction'].str.lower()
results['actual'] = results['image'].str.extract(r'(anomaly|normal)')
results.head()

Display confidence histogram

In [ ]:
(results.confidence * 100).hist(bins=20);

Display the confusion matrix

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

%matplotlib inline

confusion_matrix = pd.crosstab(results['actual'],
                               results['prediction'],
                               rownames=['Actual'],
                               colnames=['Predicted'])

sn.heatmap(confusion_matrix, annot=True, fmt='g')
plt.show()

Which images where misclassified?

In [ ]:
results[results['prediction'] != results['actual']]

Here's an idea: let's use a cleaned up version of a [random function I found on StackOverflow](https://stackoverflow.com/questions/22937589/how-to-add-noise-gaussian-salt-and-pepper-etc-to-image-in-python-with-opencv) to generate more images 😅

In [ ]:
import numpy as np

def noisy(image, noise_type, **kwargs):
    """Adds noise to an image"""
    # Get image dimensions
    row, col, chan = image.shape

    if noise_type == "gauss":
        logging.info('Adding gaussian noise (mean: %s, var: %s)', kwargs['mean'], kwargs['var'])
        gauss = np.random.normal(kwargs['mean'], kwargs['var']**0.5, (row, col, chan))
        gauss = gauss.reshape(row, col, chan)
        return image + gauss

    if noise_type == "s&p":
        logging.info('Adding salt & pepper noise (s_vs_p: %s, amount: %s)', kwargs['s_vs_p'], kwargs['amount'])
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(kwargs['amount']*image.size*kwargs['s_vs_p'])
        coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
        out[tuple(coords)] = 1
        # Pepper mode
        num_pepper = np.ceil(kwargs['amount']*image.size*(1.-kwargs['s_vs_p']))
        coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
        out[tuple(coords)] = 0
        return out

    if noise_type == "poisson":
        logging.info('Adding poisson noise')
        vals = len(np.unique(image))
        vals = 2**np.ceil(np.log2(vals))
        return np.random.poisson(image * vals) / float(vals)

    if noise_type == "speckle":
        logging.info('Adding speckle noise')
        gauss = np.random.randn(row, col, chan)
        gauss = gauss.reshape(row, col, chan)
        return image + image * gauss

    logging.error('Invalid noise type: %s', noise_type)
    logging.info('Returning original image')
    return image

While we're at it, we can also add some *flips* and *rotations*

In [ ]:
import cv2
from imutils import rotate

# Where to place the noisy images
target_folder = input('Target Folder: ') or DEFAULT_NOISY_FOLDER
if not os.path.isdir(target_folder):
    logging.info('Creating directory %s', target_folder)
    os.mkdir(target_folder)

for img in glob(images_folder + '/*'):
    # Read image
    logging.info('Processing image %s', img)
    image = cv2.imread(img)

    # Add gaussian noise
    gaussian = noisy(image, 'gauss', mean=0.0, var=1.5)

    # Add salt&pepper noise
    salt_pepper = noisy(image, 's&p', s_vs_p=0.5, amount=0.04)

    # Add poisson noise
    poisson = noisy(image, 'poisson')

    # Add speckle noise
    # Problem 1: speckled circuitboard images are too big to send via API
    speckle = noisy(image, 'speckle')
    # Problem 2: DetectAnomalies will return an error if we resize the image
    # https://qiita.com/ma2shita/items/fb61de364ab316ff47a4
    #speckle = cv2.resize(speckle, (speckle.shape[0] // 2, speckle.shape[1] // 2), interpolation = cv2.INTER_AREA)
    # Solution: Save with lower quality

    # Flip and rotate images
    logging.info("Flipping image (vertical)")
    flip_vertical = cv2.flip(image, 0)
    logging.info("Flipping image (horizontal)")
    flip_horizontal = cv2.flip(image, 1)
    logging.info("Rotating images")
    rotated = rotate(image, 5)
    
    # Save results
    logging.info('Saving images')
    cv2.imwrite(target_folder + '/gaussian_' + os.path.basename(img), gaussian)
    cv2.imwrite(target_folder + '/sp_' + os.path.basename(img), salt_pepper)
    cv2.imwrite(target_folder + '/poisson_' + os.path.basename(img), poisson)
    cv2.imwrite(target_folder + '/speckle_' + os.path.basename(img), speckle, [int(cv2.IMWRITE_JPEG_QUALITY), 75])
    cv2.imwrite(target_folder + '/flipv_' + os.path.basename(img), flip_vertical)
    cv2.imwrite(target_folder + '/fliph_' + os.path.basename(img), flip_horizontal)
    cv2.imwrite(target_folder + '/rotated_' + os.path.basename(img), rotated)

and check how the model behaves

In [ ]:
%timeit

args = [
    "--project", project_name,
    "--model", model_version,
    "--images", target_folder,
    "--save", 'results_augmented.csv'
]

l4v_app.main(lookout_vision, args)

In [ ]:
import pandas as pd

# Load and process results
aug_results = pd.read_csv('results_augmented.csv')
aug_results['prediction'] = aug_results['prediction'].str.lower()
aug_results['actual'] = aug_results['image'].str.extract(r'(anomaly|normal)')
aug_results.head()

In [ ]:
# Filter out low confidence predictions
safe_results = aug_results[aug_results['confidence'] > 0.95]

confusion_matrix = pd.crosstab(safe_results['actual'],
                               safe_results['prediction'],
                               rownames=['Actual'],
                               colnames=['Predicted'])

sn.heatmap(confusion_matrix, annot=True, fmt='g')
plt.show()

### References

* (AWS) [AWS for Industrial](https://aws.amazon.com/industrial/)
* (AWS) [Amazon Lookout for Vision – General Information](https://aws.amazon.com/lookout-for-vision/)
* (AWS) [Amazon Lookout for Vision – Developer Guide](https://docs.aws.amazon.com/lookout-for-vision/latest/developer-guide/what-is.html)
* (AWS) [Amazon Lookout for Vision – API Reference](https://docs.aws.amazon.com/lookout-for-vision/latest/APIReference/Welcome.html)
* (AWS) [AWS CLI - `lookoutvision`](https://docs.aws.amazon.com/cli/latest/reference/lookoutvision/index.html)
* (AWS) [Boto3 – `LookoutForVision`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lookoutvision.html)
* (AWS) [AWS SDK for Go - `lookoutforvision`](https://docs.aws.amazon.com/sdk-for-go/api/service/lookoutforvision/)
* (AWS) [AWS Announces Five Industrial Machine Learning Services](https://press.aboutamazon.com/news-releases/news-release-details/aws-announces-five-industrial-machine-learning-services)
* (AWS) [New ML Service Simplifies Defect Detection for Manufacturing](https://aws.amazon.com/blogs/aws/amazon-lookout-for-vision-new-machine-learning-service-that-simplifies-defect-detection-for-manufacturing/)
* (AWS) [How a ‘Think Big’ idea helped bring Lookout for Vision to Life](https://www.amazon.science/latest-news/how-a-think-big-idea-helped-bring-lookout-for-vision-to-life)
* (AWS) [Amazon Lookout for Vision – Pricing](https://aws.amazon.com/lookout-for-vision/pricing/)
* (GitHub) [Amazon Lookout for Vision Demo](https://github.com/aws-samples/amazon-lookout-for-vision-demo)
* (1Strategy) [Quick Take: Amazon Lookout for Vision](https://www.1strategy.com/blog/2020/12/07/quick-take-amazon-lookout-for-vision/)
* (YouTube) [AWS on Air 2020: What’s Next ft. Amazon Lookout for Vision](https://www.youtube.com/watch?v=fOh-p9P8TFo)
* (Provectus) [Defect Detection with Amazon Lookout for Vision](https://provectus.com/defect-detection-amazon-lookout-for-vision/)

![Assembly Line](images/assembly_line.gif)